In [ ]:
!pip install streamlit pandas numpy matplotlib seaborn scikit-learn simpy joblib pyngrok

In [ ]:
%%writefile app_streamlit.py

# --- ARQUIVO: app_streamlit.py ---
# (CORREÇÃO: Convertido anomalous_ids para list() antes de passar para a função cacheada)

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import simpy
import random
import math
import time
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from typing import Optional, List, Tuple, Dict

sns.set(style="whitegrid")

# =======================================================
# SEÇÃO 1 - TEORIA DE FILAS (M/M/c)
# =======================================================

class MMCQueue:
    def __init__(self, arrival_rate, service_rate, servers):
        self.lambda_ = arrival_rate
        self.mu = service_rate
        self.c = servers

    def utilization(self):
        if self.c == 0 or self.mu == 0:
            return float('inf') if self.lambda_ > 0 else 0.0
        return self.lambda_ / (self.c * self.mu)

    def _p0(self):
        rho = self.utilization()
        c = self.c
        if rho >= 1:
            return 0.0
        try:
            sum_terms = sum((c * rho) ** n / math.factorial(n) for n in range(c))
            last_term = ((c * rho) ** c) / (math.factorial(c) * (1 - rho))
            return 1 / (sum_terms + last_term)
        except (OverflowError, ValueError):
            return 0.0

    def avg_waiting_time_in_queue(self):
        rho = self.utilization()
        c = self.c
        if rho >= 1:
            return float('inf')
        p0 = self._p0()
        if p0 == 0.0:
             return float('inf')
        try:
            numerator = ((c * rho) ** c) * p0
            denominator = math.factorial(c) * (1 - rho) ** 2
            Lq = numerator / denominator
            return (Lq / self.lambda_) if self.lambda_ > 0 else 0.0
        except (OverflowError, ValueError):
            return float('inf')

    def avg_time_in_system(self):
        wq = self.avg_waiting_time_in_queue()
        if wq == float('inf') or self.mu == 0:
            return float('inf')
        return wq + (1 / self.mu)

# =======================================================
# SEÇÃO 2 - SIMULAÇÃO SIMPY (COM BURSTS)
# =======================================================

def estimate_arrival_rate(df):
    df = df.dropna(subset=['timestamp'])
    if len(df) < 2: return 0.0001
    ts_min, ts_max = df['timestamp'].min(), df['timestamp'].max()
    sec = (ts_max - ts_min).total_seconds()
    return len(df) / sec if sec > 0 else 0.0001

def estimate_service_time(df, mode='exp'):
    values = df['login_duration_sec'].dropna().values
    if len(values) == 0: return 1.0
    return float(np.median(values)) if mode=='exp' else values

def generate_bursts(run_time, n_bursts=2, duration=60, mult=3, seed=42):
    random.seed(seed)
    bursts = []
    if run_time <= duration:
        return []
    for _ in range(n_bursts):
        start = random.uniform(0, run_time - duration)
        bursts.append((start, start+duration, mult))
    return bursts

class AuthSystemSim:
    def __init__(self, env, servers, service_src, mode):
        self.env = env
        self.server = simpy.Resource(env, capacity=servers)
        self.service_src = service_src
        self.mode = mode
        self.wait_times = []
        self.total = 0

    def service_time(self):
        if self.mode == 'exp':
            m = float(self.service_src)
            return random.expovariate(1/m) if m > 0 else 0
        return float(np.random.choice(self.service_src))

    def process(self, _):
        arrival = self.env.now
        with self.server.request() as req:
            yield req
            self.wait_times.append(self.env.now - arrival)
            yield self.env.timeout(self.service_time())
            self.total += 1

def arrival_process(env, system, λ, runtime, bursts):
    λ_max = λ * max([m for _,_,m in bursts] + [1])
    if λ_max <= 0:
        return
    while env.now < runtime:
        interarrival = random.expovariate(λ_max)
        yield env.timeout(interarrival)
        if env.now >= runtime:
            break
        mult = 1
        for start,end,m in bursts:
            if start <= env.now <= end:
                mult = m
                break
        if random.random() <= (λ*mult)/λ_max:
            env.process(system.process(None))

@st.cache_data
def run_sim(servers, runtime, bursts_tuple, λ, m_service):
    bursts = list(bursts_tuple)
    env = simpy.Environment()
    s = AuthSystemSim(env, servers, m_service, 'exp')
    env.process(arrival_process(env, s, λ, runtime, bursts))
    env.run(until=runtime)
    return np.mean(s.wait_times or [0]), s.total

# =======================================================
# SEÇÃO 3 - FUNÇÕES DE ANÁLISE DE DADOS (PERNA 1)
# =======================================================

@st.cache_data
def load_data(path):
    df = pd.read_csv(path)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp'])
    if 'timestamp' in df.columns:
        df['hour'] = df['timestamp'].dt.hour
        df['weekday'] = df['timestamp'].dt.day_name()
    return df

def plot_bar_with_labels(df, x, y, title, ylabel='Taxa', ylim_pad=0.01, rotate_x=False):
    fig, ax = plt.subplots(figsize=(8,4))
    if df.empty or y not in df.columns:
        st.warning(f"Não há dados para plotar '{title}'")
        ax.set_title(title)
        st.pyplot(fig)
        return

    df = df.sort_values(y, ascending=False)
    bars = ax.bar(df[x].astype(str), df[y])
    min_y = df[y].min()
    max_y = df[y].max()
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_ylim(max(0, min_y - ylim_pad), max_y + ylim_pad)
    if rotate_x:
        plt.xticks(rotation=30, ha='right')
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + (ylim_pad/2 if ylim_pad>0 else 0.005), f"{height:.3f}", ha='center', fontsize=9)
    st.pyplot(fig)

def compute_kpis(df):
    kpis = {
        'total_logins': len(df),
        'success_rate': df['login_success'].mean() if 'login_success' in df.columns else None,
        'fail_rate': 1 - df['login_success'].mean() if 'login_success' in df.columns else None,
        'unique_users': df['user_id'].nunique() if 'user_id' in df.columns else None,
        'unique_ips': df['ip_address'].nunique() if 'ip_address' in df.columns else None,
        'avg_login_duration_s': df['login_duration_sec'].mean() if 'login_duration_sec' in df.columns else None
    }
    return kpis

@st.cache_data
def detect_anomalies_isolationforest(_df, features, contamination=0.01):
    df = _df.copy()
    iso = IsolationForest(contamination=contamination, random_state=42)
    X = df[features].fillna(0).values
    if len(X) == 0:
        return df, None
    iso.fit(X)
    df['anomaly_score'] = iso.decision_function(X)
    df['is_anomaly'] = (iso.predict(X) == -1).astype(int)
    return df, iso

@st.cache_data
def find_users_with_many_failures(_df, quantile_threshold=0.95, min_failures_threshold=3):
    df = _df.copy()
    if 'login_success' not in df.columns or 'user_id' not in df.columns:
        return pd.DataFrame(columns=['user_id', 'failure_count']), 0, "Colunas 'user_id' ou 'login_success' em falta."

    failures = df[df['login_success'] == 0]
    if failures.empty:
        return pd.DataFrame(columns=['user_id', 'failure_count']), 0, "Não foram registadas falhas no dataset."

    user_failures = failures.groupby('user_id').size().to_frame(name='failure_count')

    quantile_val = user_failures['failure_count'].quantile(quantile_threshold)
    final_threshold = max(quantile_val, min_failures_threshold)

    anomalous_users = user_failures[user_failures['failure_count'] >= final_threshold].sort_values(by='failure_count', ascending=False)
    return anomalous_users, final_threshold, None

@st.cache_data
def get_anomalous_user_details(_df, anomalous_user_ids_list): # O argumento agora espera uma lista
    """
    Filtra o dataframe principal para obter os detalhes de atividade
    dos utilizadores anómalos.
    """
    if 'ip_address' not in _df.columns or 'device_type' not in _df.columns:
        return pd.DataFrame(), pd.DataFrame(), "Faltam 'ip_address' ou 'device_type' no dataset."

    # Filtra toda a atividade (sucessos e falhas) dos utilizadores suspeitos
    user_activity = _df[_df['user_id'].isin(anomalous_user_ids_list)]
    if user_activity.empty:
        return pd.DataFrame(), pd.DataFrame(), "Nenhuma atividade encontrada para estes utilizadores."

    # 1. IPs mais usados por este grupo (Top 10)
    top_ips = user_activity['ip_address'].value_counts().head(10).to_frame(name='Nº de Logins (Total)')

    # 2. Dispositivos mais usados por este grupo (Top 10)
    top_devices = user_activity['device_type'].value_counts().head(10).to_frame(name='Nº de Logins (Total)')

    return top_ips, top_devices, None # None para "sem erro"

@st.cache_data
def cluster_logins(_df, n_clusters=3):
    df = _df.copy()
    features = [c for c in ['login_duration_sec', 'latency_ms'] if c in df.columns]
    if not features:
        return df, None, "Sem features (duração/latência) para clusterizar."

    data_to_cluster = df[features].fillna(df[features].median())
    if len(data_to_cluster) < n_clusters:
        return df, None, "Menos amostras do que clusters."

    X = data_to_cluster.values
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(Xs)
    df['cluster'] = labels
    return df, kmeans, f"Clusterizado com base em: {', '.join(features)}"

@st.cache_data
def train_predictive_model(_df):
    df = _df.copy()
    feats = [c for c in ['login_duration_sec', 'latency_ms'] if c in df.columns]

    if not feats or 'login_success' not in df.columns:
        return None, "Faltam 'login_success' ou features (duração/latência)."

    data = df.copy().dropna(subset=feats + ['login_success'])
    if len(data) < 10:
        return None, "Poucos dados (após dropna) para treinar."

    X = data[feats].values
    y = data['login_success'].astype(int).values

    if len(np.unique(y)) < 2:
        return None, "Modelo requer duas classes (sucesso/falha) nos dados."

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    model = LogisticRegression(max_iter=1000).fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    return model, {'accuracy': acc, 'features': feats}


# =======================================================
# SEÇÃO 4 - INTERFACE STREAMLIT (UI COMPLETA)
# =======================================================

st.set_page_config(layout="wide")
st.title("🔐 Dashboard Completo: Análise e Simulação de Login 2FA")

# --- BARRA LATERAL (SIDEBAR) ---
with st.sidebar:
    st.header("1. Carga de Dados")
    path = st.text_input("Caminho do CSV:", "/content/simulacao_login_dataset1_processed.csv")

    st.header("2. Perna 1: Análise de Dados")
    contamination = st.slider("Contaminação (Anomalias Numéricas)", 0.001, 0.1, 0.01, step=0.001, format="%.3f")
    n_clusters = st.slider("Número de clusters (KMeans)", 2, 8, 3)

    min_failures = st.number_input("Anomalia: Mín. Falhas/Utilizador", min_value=1, max_value=50, value=5)

    st.header("3. Perna 2: Modelagem de Filas")
    st.subheader("Teoria M/M/c")
    servers_m = st.slider("Servidores (Teoria):", 1, 20, 3)

    st.subheader("Simulação SimPy (Estresse)")
    servers_test = st.multiselect("Servidores para Simular:", [1, 2, 3, 4, 5, 6, 7, 8], default=[1, 2, 3, 4])
    n_bursts = st.slider("Nº de bursts (picos):", 0, 10, 3)
    mult_bursts = st.slider("Multiplicador dos bursts:", 1.0, 2000.0, 10.0, step=10.0)
    dur_bursts = st.slider("Duração bursts (s):", 30, 600, 120, step=30)
    runtime = st.number_input("Duração total da simulação (s):", value=3600, min_value=600, step=300)

# --- CARGA DOS DADOS ---
try:
    df = load_data(path)
except FileNotFoundError:
    st.error(f"Arquivo não encontrado em '{path}'. Faça o upload do seu CSV para o Colab (no ícone de pasta 📁 à esquerda).")
    st.stop()
except Exception as e:
    st.error(f"Erro ao carregar CSV: {e}")
    st.stop()


# =======================================================
# ABA 1: PERNA 1 - ANÁLISE DE DADOS
# =======================================================
st.header("PERNA 1: Análise de Ciência de Dados (O que aconteceu?)")

# --- KPIs ---
st.subheader("Resumo do dataset (KPIs)")
kpis = compute_kpis(df)
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total de logins", kpis['total_logins'])
col2.metric("Taxa de sucesso", f"{kpis['success_rate']:.2%}" if kpis['success_rate'] is not None else "N/A")
col3.metric("Utilizadores únicos", kpis['unique_users'] if kpis['unique_users'] is not None else "N/A")
col4.metric("Tempo Médio Login", f"{kpis['avg_login_duration_s']:.3f} s" if kpis['avg_login_duration_s'] is not None else "N/A")

# --- Gráficos de Barras (Taxa de Sucesso) ---
st.subheader("📊 Taxas de Sucesso por Categoria")
col1, col2 = st.columns(2)
with col1:
    if 'device_type' in df.columns and 'login_success' in df.columns:
        device_stats = df.groupby('device_type')['login_success'].agg(['mean','count']).reset_index().rename(columns={'mean':'success_rate'})
        plot_bar_with_labels(device_stats, 'device_type', 'success_rate', "Taxa de Sucesso por Dispositivo")
    else:
        st.info("Faltam colunas 'device_type' ou 'login_success' para este gráfico.")
with col2:
    if 'auth_method' in df.columns and 'login_success' in df.columns:
        auth_stats = df.groupby('auth_method')['login_success'].agg(['mean','count']).reset_index().rename(columns={'mean':'success_rate'})
        plot_bar_with_labels(auth_stats, 'auth_method', 'success_rate', "Taxa de Sucesso por Método", rotate_x=True)
    else:
        st.info("Faltam colunas 'auth_method' ou 'login_success' para este gráfico.")

# --- Histogramas (Latência e Duração) ---
st.subheader("⏱ Distribuição de Latência e Duração")
col1, col2 = st.columns(2)
with col1:
    if 'latency_ms' in df.columns:
        fig, ax = plt.subplots(figsize=(8,4))
        df['latency_ms'].plot(kind='hist', bins=50, ax=ax)
        ax.set_title("Distribuição de Latência (ms)")
        ax.set_xlabel("Latência (ms)")
        st.pyplot(fig)
    else:
        st.info("Falta a coluna 'latency_ms' para este gráfico.")
with col2:
    if 'login_duration_sec' in df.columns:
        fig, ax = plt.subplots(figsize=(8,4))
        df['login_duration_sec'].plot(kind='hist', bins=50, ax=ax)
        ax.set_title("Distribuição de Duração do Login (s)")
        ax.set_xlabel("Duração (s)")
        st.pyplot(fig)
    else:
        st.info("Falta a coluna 'login_duration_sec' para este gráfico.")

# --- Análises de ML (Anomalias, Clustering, Predição) ---
st.subheader("🤖 Modelos de ML: Anomalias e Clustering")
col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("**⚠️ Anomalias (Comportamento)**")
    af_features = [c for c in ['latency_ms','login_duration_sec'] if c in df.columns]
    if af_features:
        df_anom, iso = detect_anomalies_isolationforest(df, af_features, contamination=contamination)
        n_anom = int(df_anom['is_anomaly'].sum())
        st.write(f"Anomalias detetadas: **{n_anom}** ({contamination*100:.1f}%)")
        if n_anom > 0:
            st.dataframe(df_anom[df_anom['is_anomaly']==1][af_features + ['is_anomaly']].head())
    else:
        st.info("Faltam colunas (latência/duração) para detetar anomalias.")

with col2:
    st.markdown("**🔎 Clusterização (KMeans)**")
    df_clustered, kmeans_model, cluster_info = cluster_logins(df, n_clusters=n_clusters)
    if kmeans_model:
        cluster_features = [f for f in ['login_duration_sec','latency_ms'] if f in df_clustered.columns]
        st.write("Centros dos clusters (medianas):")
        st.dataframe(df_clustered.groupby('cluster')[cluster_features].median().round(3))
        st.write("Contagem por cluster:")
        st.dataframe(df_clustered['cluster'].value_counts().sort_index().to_frame(name="count"))
    else:
        st.info(f"Não foi possível clusterizar: {cluster_info}")

with col3:
    st.markdown("**🤖 Modelo Preditivo (Sucesso)**")
    model, model_info = train_predictive_model(df)
    if model:
        st.write(f"Acurácia do modelo: **{model_info['accuracy']:.2%}**")
        st.write(f"Features usadas: {', '.join(model_info['features'])}")
    else:
        st.info(f"Não foi possível treinar modelo: {model_info}")

st.divider()

# --- SECÇÃO DE UI DE ANÁLISE DE FALHAS (ATUALIZADA) ---
st.subheader("🛡️ Análise de Segurança: Utilizadores com Múltiplas Falhas (Bots/Ataques)")

anomalous_users_df, threshold, error_msg = find_users_with_many_failures(
    df,
    min_failures_threshold=min_failures
)

st.write(f"A detetar utilizadores com **{min_failures}** ou mais falhas (ou que estejam no top 5% de falhas, o que for maior).")

if error_msg:
    st.info(error_msg)
elif anomalous_users_df.empty:
    st.info("Nenhum utilizador encontrado com um número anómalo de falhas (acima do limiar).")
else:
    st.write(f"Encontrados **{len(anomalous_users_df)}** utilizadores anómalos (limiar de {int(threshold)} falhas):")

    col1, col2 = st.columns([1, 2])

    with col1:
        st.write("Top 10 Utilizadores Suspeitos:")
        st.dataframe(anomalous_users_df.head(10))

    with col2:
        st.write("Gráfico: Top 10 Falhas por Utilizador")
        chart_data = anomalous_users_df.head(10).reset_index()
        st.bar_chart(chart_data, x='user_id', y='failure_count')

    st.markdown("---")
    st.markdown("#### Investigação: O que estes utilizadores suspeitos têm em comum?")

    # Obtém os IDs dos utilizadores suspeitos
    anomalous_ids = anomalous_users_df.index

    # --- A CORREÇÃO ESTÁ AQUI ---
    # Convertemos o objeto 'Index' (que não é "hasheável") para uma 'list' (que é)
    anomalous_ids_list = list(anomalous_ids)

    # Chamamos a função cacheada com a lista, e não com o Index
    top_ips, top_devices, details_error = get_anomalous_user_details(df, anomalous_ids_list)
    # --- FIM DA CORREÇÃO ---

    if details_error:
        st.info(details_error)
    else:
        col1_details, col2_details = st.columns(2)

        with col1_details:
            st.write("Top IPs Usados por este Grupo:")
            st.dataframe(top_ips)
            if not top_ips.empty and top_ips.iloc[0, 0] > len(anomalous_ids):
                st.warning("ALERTA: Um único IP está a ser usado para atacar múltiplas contas!")

        with col2_details:
            st.write("Top Dispositivos Usados por este Grupo:")
            st.dataframe(top_devices)

st.divider()

# =======================================================
# ABA 2: PERNA 2 - SIMULAÇÃO DE FILAS
# =======================================================
st.header("PERNA 2: Simulação Computacional (O que acontecerá?)")

# --- PARÂMETROS GLOBAIS ---
try:
    arrival_rate = estimate_arrival_rate(df)
    service_time = estimate_service_time(df)
    service_rate = 1/service_time
except Exception as e:
    st.error(f"Erro ao estimar parâmetros do dataset: {e}")
    st.stop()

st.subheader("Parâmetros Globais Estimados (Usados nas simulações)")
col1, col2 = st.columns(2)
col1.metric("λ (Taxa de Chegada)", f"{arrival_rate:.4f} chegadas/s")
col2.metric("μ (Taxa de Serviço)", f"{service_rate:.4f} atend./s", f"(Tempo médio: {service_time:.3f} s)")

st.divider()

# --- ANÁLISE TEÓRICA ---
st.subheader("📈 Teoria M/M/c (Cenário de Média Estável)")
st.write(f"Análise teórica para **{servers_m}** servidores, assumindo fluxo constante e sem picos.")
q = MMCQueue(arrival_rate, service_rate, servers_m)
rho = q.utilization()

col1, col2, col3 = st.columns(3)
col1.metric(f"Servidores (c)", servers_m)
col2.metric("Utilização (ρ)", f"{rho:.3%}")
if rho >= 1.0:
    col3.metric("Espera Média (Wq)", "∞ (Instável)")
    st.warning(f"Com {servers_m} servidores, o sistema teórico é instável (ρ ≥ 1).")
else:
    col3.metric("Espera Média (Wq)", f"{q.avg_waiting_time_in_queue():.4f} s")

st.divider()

# --- SIMULAÇÃO PRÁTICA ---
st.subheader("🔬 Simulação com Sobrecarga (Bursts)")
st.write("Análise prática que simula picos de tráfego (bursts) para ver o impacto real no tempo de espera.")

if st.button("▶️ Executar Simulação de Estresse", type="primary"):

    bursts = generate_bursts(runtime, n_bursts, dur_bursts, mult_bursts)
    bursts_tuple = tuple(bursts) # Converte para tupla para o cache

    st.write(f"Simulando {len(servers_test)} cenários com {n_bursts} picos de {mult_bursts}x...")

    with st.spinner(f"Rodando simulação... Isso pode levar alguns segundos."):
        results = []
        start_time = time.time()

        for s in servers_test:
            w, tot = run_sim(s, runtime, bursts_tuple, arrival_rate, service_time)
            results.append([s, w, tot])

        end_time = time.time()
        st.success(f"Simulação concluída em {end_time - start_time:.2f} segundos.")

    res_df = pd.DataFrame(results, columns=["Servidores","Espera_Média_s","Total_Processado"])
    res_df = res_df.set_index("Servidores")

    st.write("Resultados da Simulação (com Bursts):")
    st.dataframe(res_df.style.format("{:.4f}"))

    st.write("Tempo Médio de Espera vs. Servidores")
    chart_df = res_df.reset_index()

    st.line_chart(chart_df, x="Servidores", y="Espera_Média_s")

st.caption("Fim do Dashboard.")

Overwriting app_streamlit.py


In [ ]:
from pyngrok import ngrok
from google.colab import userdata
import os

# Encerra túneis ngrok anteriores (se houver) para evitar erros
try:
    ngrok.kill()
except:
    pass

# Get the authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("ngrok authtoken set.")
else:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")


# Para o streamlit em segundo plano

!streamlit run app_streamlit.py &> /dev/null &

# Abre um túnel público para a porta 8501 (padrão do streamlit)
# Use connect() without specifying the authtoken again, as it's set globally now
public_url = ngrok.connect(8501).public_url

print(f"🎉 Seu app Streamlit COMPLETO está pronto! Acesse aqui: {public_url}")

ngrok authtoken set.
🎉 Seu app Streamlit COMPLETO está pronto! Acesse aqui: https://nonalienating-grushie-despina.ngrok-free.dev
